In [1]:
from dotenv import load_dotenv
import os
import json

def configure():
    load_dotenv()
configure()

In [ ]:
import requests

url = 'https://api.sankhya.com.br/login'

headers = {
    'Appkey': os.getenv('Appkey'),
    'Token': os.getenv('Token'),
    'Username': os.getenv('Integrador'),
    'Password': os.getenv('Password')
}

response = requests.post(url, headers=headers)

print(response.status_code)
print(response.text)

bearer = response.json()['bearerToken']
bearer


In [ ]:
import requests
import json

url = "https://api.sankhya.com.br/gateway/v1/mge/service.sbr?serviceName=CRUDServiceProvider.loadRecords&outputType=json"

headers = {
  'Authorization': f'Bearer {bearer}',
  'Appkey': os.getenv('Appkey'),
  'Token': os.getenv('Token'),
  'Username': f'{os.getenv('Username')}@vitalehospitalar.com.br',
  'Password': os.getenv('Password'),
  'Content-Type': 'application/json',
}


offset_page = 0
has_more_result = True
resultados_completos = []

while has_more_result:
    payload = json.dumps({
      "serviceName": "CRUDServiceProvider.loadRecords",
      "requestBody": {
        "dataSet": {
          "rootEntity": "Produto",
          "includePresentationFields": "N",
          "tryJoinedFields": "true",
          "offsetPage": str(offset_page),
          "entity": [
            {
              "path": "",
              "fieldset": {
                "list": "DESCRPROD, CODPROD, MARCA, NCM, COMPLDESC, AD_REGISTRO_MS, ATIVO, REFFORN, AD_CATEGORIA, AD_SUBCATEGORIA, AD_LINHA, USOPROD"
              }
            },
            {
              "path": "GrupoProduto",
              "fieldset": {
                "list": "DESCRGRUPOPROD"
              }
            }
          ]
        }
      }
    })
    
    response_produto = requests.request("GET", url, headers=headers, data=payload)
    response_data = response_produto.json()
    print(response_data)  # Verifica toda a resposta
    print(f"Página: {offset_page}, mais páginas a serem extraídas? {response_data['responseBody']['entities']['hasMoreResult']}")  # Verifica a página atual 
    


    
    # Adiciona os dados da resposta à lista de resultados completos
    resultados_completos.append(response_data['responseBody']['entities'])
    
    ### Verifica se há mais resultados e incrementa o offsetPage
    if response_data['responseBody']['entities']['hasMoreResult'] == "true":
        offset_page += 1
    else:
        has_more_result = False



In [ ]:
import pandas as pd

# Combinar todas as entidades de cada página em uma única lista
all_entities = []
for page_data in resultados_completos:
    all_entities.extend(page_data['entity'])

# Como o metadata dos campos é o mesmo para todas as páginas, podemos usar o metadata da primeira página
fields = [field['name'] for field in resultados_completos[0]['metadata']['fields']['field']]

# Extraindo os dados das colunas para o DataFrame
data = [
    [item.get('f' + str(index), {}).get('$', None) for index in range(len(fields))]
    for item in all_entities
]

# Criar o DataFrame
df_produto = pd.DataFrame(data, columns=fields)
# Filtrar as linhas que possuem o valor 'S' na coluna USOPROD
df_produto = df_produto[df_produto['USOPROD'] != 'S']
# Excluir coluna USOPROD
df_produto = df_produto.drop(columns=['USOPROD'])
# Remover espaços em branco no início e no final de todos os valores de string
df_produto = df_produto.map(lambda x: x.strip() if isinstance(x, str) else x)
# Substituir os valores 'S' e 'N' por 'Sim' e 'Não'
df_produto['ATIVO'].replace({'S': 'Sim', 'N': 'Não'}, inplace=True)
# Substituir os valores 'EP' e 'Ter. Ablativas' por 'ELETROFISIOLOGIA' e 'TERAPIAS ABLATIVAS'
df_produto['AD_LINHA'].replace({'EP': 'ELETROFISIOLOGIA', 'TER. ABLATIVAS': 'TERAPIAS ABLATIVAS'}, inplace=True)

df_produto = df_produto.rename(columns={'DESCRPROD': 'Descrição',
                                        'CODPROD': 'Código',
                                        'MARCA': 'Marca',
                                        'COMPLDESC': 'Descrição Generica',
                                        'AD_REGISTRO_MS': 'Registro M.S.',
                                        'ATIVO': 'Ativo',
                                        'REFFORN': 'Referência do Fornecedor',
                                        'AD_CATEGORIA': 'Categoria',
                                        'AD_SUBCATEGORIA': 'Sub categoria',
                                        'AD_LINHA': 'Linha',
                                        'GrupoProduto_DESCRGRUPOPROD': 'Descrição (Grupo Produto)',
                                        })
ordem_colunas = ['Descrição', 'Código', 'Marca', 'NCM',
                 'Descrição Generica', 'Registro M.S.', 'Ativo',
                 'Referência do Fornecedor', 'Categoria', 'Sub categoria',
                 'Descrição (Grupo Produto)', 'Linha']
df_produto = df_produto[ordem_colunas]


df_produto.to_csv(r"Y:\Vitale\Automação_Extração_Relatórios\resultado.csv", index=False, sep=';', encoding='utf-8-sig')
